## Day 24

https://adventofcode.com/2020/day/24

In [1]:
import collections
import enum
import operator
import typing

In [2]:
import aocd

In [3]:
TEST_DATA = """
sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
"""

In [4]:
class HexCubeCoordinates(enum.Enum):
    """See cube coordinates:
        https://www.redblobgames.com/grids/hexagons/#coordinates
    """
    E = (1, -1, 0)
    SE = (0, -1, 1)
    SW = (-1, 0, 1)
    W = (-1, 1, 0)
    NW = (0, 1, -1)
    NE = (1, 0, -1)
    
    @classmethod
    def from_str(cls, s):
        try:
            return cls[s.upper()]
        except KeyError:
            return None
        
    def move(self, gridpoint):
        return tuple(map(operator.add, gridpoint, self.value))
    
    @staticmethod
    def adjacent(gridpoint):
        for direction in HexCubeCoordinates:
            yield direction.move(gridpoint)

In [5]:
def parse_directions(line: str):
    move = ''
    for c in line:
        move += c
        assert len(move) <= 2
        direction = HexCubeCoordinates.from_str(move)
        if direction is not None:
            yield direction
            move = ''

In [6]:
def parse_data(data):
    for line in data.splitlines():
        if line:
            yield [d for d in parse_directions(line)]

### Solution to Part 1

In [7]:
def flip(tile, flipped_tiles):
    if tile in flipped_tiles:
        flipped_tiles.discard(tile)
    else:
        flipped_tiles.add(tile)

In [8]:
def process_moves(data):
    flipped_tiles = set()
    for line in parse_data(data):
        tile = (0, 0, 0)
        for direction in line:
            tile = direction.move(tile)
        flip(tile, flipped_tiles)
    return flipped_tiles

In [9]:
flipped_tiles = process_moves(aocd.get_data(day=24, year=2020))
# flipped_tiles = process_moves(TEST_DATA)
len(flipped_tiles)

269

### Solution to Part 2

In [10]:
def count_flipped_neighbors(tile, flipped_tiles):
    return sum(
        neighbor in flipped_tiles
        for neighbor in HexCubeCoordinates.adjacent(tile)
    )

In [11]:
def flipped_tiles_to_flip(flipped_tiles):
    for tile in flipped_tiles:
        n = count_flipped_neighbors(tile, flipped_tiles)
        if n == 0 or n > 2:
            yield tile

In [12]:
def get_unflipped_candidates(flipped_tiles):
    return set([
        neighbor
        for tile in flipped_tiles
        for neighbor in HexCubeCoordinates.adjacent(tile)
        if neighbor not in flipped_tiles
    ])

In [13]:
def unflipped_tiles_to_flip(flipped_tiles):
    for tile in get_unflipped_candidates(flipped_tiles):
        n = count_flipped_neighbors(tile, flipped_tiles)
        if n == 2:
            yield tile

In [14]:
def turn(flipped_tiles):
    new_tiles = flipped_tiles.copy()
    for tile in flipped_tiles_to_flip(flipped_tiles):
        flip(tile, new_tiles)
    for tile in unflipped_tiles_to_flip(flipped_tiles):
        flip(tile, new_tiles)
    return new_tiles

In [ ]:
part2 = flipped_tiles.copy()
for _ in range(100):
    part2 = turn(part2)
len(part2)